# MEASUREMENT

See [measurement](https://ohdsi.github.io/CommonDataModel/cdm54.html#measurement). 

This table contains the records of measurements or tests performed on the patient. It contains both the measurement order and the measurement results. It has a key:value structure. Here goes any “observation” of the patient for which it was necessary to perform a test. 

```{mermaid}
erDiagram
    OMOP_MEASUREMENT {
        integer measurement_id
        integer person_id
        integer measurement_concept_id
        date measurement_date
        datetime measurement_datetime
        varchar(10) datetime
        integer measurement_type_concept_id
        integer operator_concept_id
        float value_as_number
        integer value_as_concept_id
        integer unit_concept_id
        float range_low
        float range_high
        integer provider_id
        integer visit_occurrence_id
        integer visit_detail_id
        varchar(50) measurement_source_value
        integer measurement_source_concept_id
        varchar(50) unit_source_value
        integer unit_source_concept_id
        varchar(50) value_source_value
        integer measurement_event_id
        integer meas_event_field_concept_id
    }
```

The execution of the transformation is carried out by the file [genomop_measurement.py](../examples/genomop_measurement.py). 

This script performs the following steps:
1. Loads parameters
2. Loads vocabulary tables
   - In this case it also loads the CLC codes to map the measurement codes in the MPA module of BPS. 
3. Loads each file
   1. Assign a new `vocabulary_id` column if needed. See `append_vocabulary` parameter.
   2. Rename the column containing the conditions names or codes to `source_value`. See `column_map` parameter.
   3. Maps source codes to a specific column in the CONCEPT table to retrieve the `source_concept_id`. See `vocabulary_config` parameter.
4. Creates a single table with all files.
5. Ensure numeric values are numeric.
6. Tries to map measurement units. See `create_vocabulary_mapping()` in module [general.py](../bps_to_omop/general.py) and `map_measurement_units()` in module [measurement.py](../bps_to_omop/measurement.py),
   - If some mappings cannot be performed, a warning will be prompted.
7. Map source_concept_id codes to concept_id. 
   - Both `measurement_source_concept_id` and `unit_source_concept_id` will be mapped.
8. Check for codes that were not mapped.
   - Parameters `unmapped_measurement` and `unmapped_unit` can be used to define additional mappings to be applied for measurement codes and units, respectively.
9. Creates primary key (`measurement_id`)
10. Finds any entries that are contained within a visit in the VISIT_OCCURRENCE table and assigns the corresponding `visit_occurrence_id`.
11. Adapt the table to the schema of the CONDITION_OCCURRENCE table.
12. Saves the omop table to the defined output folder.
 

The configuration file will be [genomop_measurement_params.yaml](../examples/genomop_measurement_params.yaml).